In [1]:

from transformers import pipeline
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sys
sys.path.append("../CODE-Baseline")  
import warnings
warnings.filterwarnings('ignore')


from salary_baseline import extract_salary_with_inference


file_path = '../DATASETS/salary_labelled_development_set.csv'
test_file_path = '../DATASETS/salary_labelled_test_set.csv'
df = pd.read_csv(file_path)

model_name = "deepset/bert-base-cased-squad2"

tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer
)

/Users/garry/vs_code/6713/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Development dataset:
Precision: 0.7811
Recall: 0.9370
F1 Score: 0.8519
Accuracy: 0.8489

Test dataset:
Precision: 0.7412
Recall: 0.9206
F1 Score: 0.8212
Accuracy: 0.8219


Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


In [2]:
# html -> context 
def clean_html_tags(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    for tag in soup(["script", "style"]):
        tag.decompose()
    text = soup.get_text(separator=" ", strip=True)
    return text

In [3]:
salary_question = "what is fixed salary or salary range?"
pay_freq_question = "Is the salary paid monthly, hourly, yearly, weekly, or daily?"

def get_salary_using_nFT_Bert(text,nation_code):
  context=clean_html_tags(text)
  salary_result = qa_pipeline({"context": context, "question": salary_question})
  t_result=extract_salary_with_inference(salary_result["answer"],nation_code)
  # print(t_result)
  if t_result!='0-0-None-None':
    pay_freq_result = qa_pipeline({"context": context, "question": pay_freq_question})
    # print(pay_freq_result["answer"])
    
  return t_result

    
    
    
df['predicted_salary'] = df.apply(
    lambda row: get_salary_using_nFT_Bert(
        f"{row['job_title']} {row['job_ad_details']}",
        row['nation_short_desc']
    ),
    axis=1
)
    


In [ ]:
# TP, FP, TN, FN
TP = np.sum((df['predicted_salary'] == df['y_true']) & (df['y_true'] != "0-0-None-None"))
FP = np.sum((df['predicted_salary'] != df['y_true']) & (df['predicted_salary'] != "0-0-None-None"))
FN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] != "0-0-None-None"))
TN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] == "0-0-None-None"))

precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (TP + TN) / (FP + FN + TP + TN)

# Print prediction vs ground truth
# print("\n🔍 Prediction vs Ground Truth:\n")
# for i, row in df.iterrows():
#     predicted = row['predicted_salary']
#     expected = row['y_true']
#     if predicted != expected:
#         print(f"[{i}] ❌ Predicted: {predicted} | Expected: {expected}")
#         print(f"{row['job_id']} {row['job_title']} {row['job_ad_details']}")
#         print()
    # else:
    #     print(f"[{i}] ✅ Matched:   {predicted}")

print("Development dataset:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print()

In [ ]:
# test set
df = pd.read_csv(test_file_path)
df['predicted_salary'] = df.apply(
    lambda row: get_salary_using_nFT_Bert(
        f"{row['job_title']} {row['job_ad_details']}",
        row['nation_short_desc']
    ),
    axis=1
)

TP = np.sum((df['predicted_salary'] == df['y_true']) & (df['y_true'] != "0-0-None-None"))
FP = np.sum((df['predicted_salary'] != df['y_true']) & (df['predicted_salary'] != "0-0-None-None"))
FN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] != "0-0-None-None"))
TN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] == "0-0-None-None"))


precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (TP + TN) / (FP + FN + TP + TN)

print("Test dataset:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")